In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import numpy as np
import torch as th
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

In [5]:
root = './data'
download = False # download MNIST dataset or not
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = dset.MNIST(root=root, train=True, transform=trans, download=download)
test_set = dset.MNIST(root=root, train=False, transform=trans)

batch_size = 100

train_loader = th.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = th.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
shuffle=False)

In [6]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def classify(self, x):
        x = self.forward(x)
        x = x.data.numpy()
        return np.argmax(x, axis=1)


In [7]:
model = LeNet()

In [10]:
optimizer = optim.Adagrad(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
epochs = 10

In [11]:
for it in range(epochs):
    for batch_idx, (x, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            x = Variable(x)
            labels = Variable(labels)
            out = model(x)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
            if (batch_idx + 1) % 100 == 0:
                print(f'Loss at {batch_idx + 1} was {loss.data[0]}')
            
    

Loss at 100 was 0.21950587630271912
Loss at 200 was 0.022060111165046692
Loss at 300 was 0.04995805770158768
Loss at 400 was 0.09831493347883224
Loss at 500 was 0.03801900893449783
Loss at 600 was 0.0161729883402586
Loss at 100 was 0.016481755301356316
Loss at 200 was 0.0308665968477726
Loss at 300 was 0.05004104599356651
Loss at 400 was 0.023686721920967102
Loss at 500 was 0.01133345253765583
Loss at 600 was 0.04204659163951874
Loss at 100 was 0.037173300981521606
Loss at 200 was 0.0443379245698452
Loss at 300 was 0.02301931567490101
Loss at 400 was 0.00807615090161562
Loss at 500 was 0.008216501213610172
Loss at 600 was 0.04570220410823822
Loss at 100 was 0.007197526749223471
Loss at 200 was 0.028109576553106308
Loss at 300 was 0.010770407505333424
Loss at 400 was 0.02896406129002571
Loss at 500 was 0.013758838176727295
Loss at 600 was 0.027859685942530632
Loss at 100 was 0.03592894971370697
Loss at 200 was 0.010574831627309322
Loss at 300 was 0.014562135562300682
Loss at 400 was 0.0

In [44]:
evaluate_x = Variable(test_loader.dataset.test_data.type_as(th.FloatTensor()))
evaluate_y = Variable(test_loader.dataset.test_labels.type_as(th.LongTensor()))

In [45]:
evaluate_x = evaluate_x.view(len(evaluate_x), 1, 28, 28)

In [46]:
output = model.classify(evaluate_x)

In [47]:
metrics.accuracy_score(output, evaluate_y.data.numpy())

0.98939999999999995